# EM with Gaussian Mixture Models

## Imports

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, chi2

## Color cycle
Just for consistent visualization we pick the default matplotlib color cycle

In [2]:
default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

## Expectation step
In EM for GMM the expectation step consist in computing the responsibilities r_ik

In [3]:

def compute_responsibility(data, pi, mu, sigma):
    n_classes = len(pi)

    # Compute the numerator
    unnormalized_resposibility = list()
    for k in range(n_classes):
        p_x_theta = multivariate_normal.pdf(data, mean=mu[k], cov=sigma[k])

        r_ik_unnormalized = pi[k] * p_x_theta
        unnormalized_resposibility.append(r_ik_unnormalized)

    unnormalized_resposibility = np.stack(unnormalized_resposibility)

    # Divide by the sum over all possible classes
    r_ik = unnormalized_resposibility / unnormalized_resposibility.sum(axis=0)

    return r_ik

## Maximization Step
Now, given the responsibilities, the maximization step can be computed in closed form. Therefore we only need to compute the parameter update!

In [4]:
def update_parameters(data, r_i):
    n_classes = len(r_i)
    N = data.shape[0]

    r_sum = r_i.sum(axis=1)

    pi_new = list()
    mu_new = list()
    sigma_new = list()

    for k in range(n_classes):
        # Update class weights
        pi_new_k = r_sum[k]/N

        # Update mean
        mu_new_k = r_i[k]@data/r_sum[k]

        # Update covariance
        delta_k = data - mu_new_k
        sigma_new_k = delta_k.T @ np.diag(r_i[k]/r_sum[k]) @ delta_k

        pi_new.append(pi_new_k)
        mu_new.append(mu_new_k)
        sigma_new.append(sigma_new_k)

    return np.array(pi_new), np.stack(mu_new), np.stack(sigma_new)

## Visualization functions
Here we have some visualization functions.

1. get_clusters identify the class with higher responsibility for each datapoint
2. plot_dataset plots the raw dataset and the initial clusters
3. plot_mixture is similar to plot data, but highlights the datapoints with colors related to the most important cluster

In [5]:
def get_clusters(data, responsibility):
    n_classes = len(responsibility)

    # Associate datapoints to clusters, by selecting the highest responsibility
    cluster_idx = np.argmax(responsibility, axis=0)

    clusters = list()

    # Split the data into cluster lists
    for k in range(n_classes):
        cluster_k = data[cluster_idx == k]
        clusters.append(cluster_k)

    return clusters

def plot_dataset(data, mu, sigma):
    n_clusters = len(mu)
    plt.figure()

    # Plot the raw data
    plt.plot(data[:, 0], data[:, 1], 'g.')

    # Plot the mixture model
    for k in range(n_clusters):
        plot_mixture(mu[k], sigma[k], default_colors[k])

    # Set properly axis limits and aspect ratio
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.gca().set_aspect('equal')
    plt.tight_layout()

    plt.show()


def plot_mixture(mu, sigma, color, n_points=100):

    # Find eigenvalues and eigenvectors of the covariance matrix
    vals, vecs = np.linalg.eigh(sigma)
    order = vals.argsort()[::-1]
    vals, vecs = vals[order], vecs[:, order]

    # Compute the rotation angle
    theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))

    # Compute the scaling factor for each axis
    chi = chi2.ppf(0.95, df=2)
    width, height = 2 * np.sqrt(chi * vals)

    # Generate the points of the ellipse and rotate them
    t = np.linspace(0, 2*np.pi, n_points)
    ellipse = np.column_stack([width/2 * np.cos(t), height/2 * np.sin(t)])
    rot = np.array([[np.cos(np.radians(theta)), -np.sin(np.radians(theta))],
                    [np.sin(np.radians(theta)), np.cos(np.radians(theta))]])
    ellipse = ellipse @ rot.T + mu

    # Plot the ellipse
    plt.plot(ellipse[:,0], ellipse[:,1], color=color)[0]
    plt.plot(mu[0], mu[1], 'x', color=color)


def plot_cluster(clusters, mu, sigma):
    plt.figure()

    # plot the data separated in clusters and the mixtures
    for k, cluster in enumerate(clusters):
        plt.plot(cluster[:, 0], cluster[:, 1], '.', color=default_colors[k])
        plot_mixture(mu[k], sigma[k], default_colors[k])

    # Set properly axis limits and aspect ratio
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.gca().set_aspect('equal')
    plt.tight_layout()

    plt.show()


## Main Loop
Now we can run the EM algorithm following the three steps:
1. Initialization
2. Expectation
3. Maximization

Here we set a simplified convergence criterion: if the mean doesn't change much between two steps, terminate the algorithm. More advance criterion can be used instead.

In [ ]:
import os
print(os.getcwd())

In [ ]:
# Max iterations
N = 20

# Convergence threshold
epsilon = 1e-3

# Load dataset
data = np.load("old_faithful_rescaled.npy")
xlim = [-2.0, 2.0]
ylim = [-2.0, 2.0]

In [ ]:
# Initialize parameters
pi_0 = 0.5
mu_0 = np.array([1., 0])
sigma_0 = 0.1*np.eye(2)

pi_1 = 0.5
mu_1 = np.array([-1., 0])
sigma_1 = 0.1*np.eye(2)

In [ ]:
# Transform everything in a single array
pi =  np.array([pi_0, pi_1])
mu = np.stack([mu_0, mu_1])
sigma = np.stack([sigma_0, sigma_1])

In [ ]:
# Plot dataset
plot_dataset(data, mu, sigma)

In [ ]:
# EM loop
for it in range(N):
    old_mu = mu.copy()
    old_sigma = sigma.copy()
    old_pi = pi.copy()

    # Expectation
    responsibility = compute_responsibility(data, pi, mu, sigma)

    # Maximization
    pi, mu, sigma = update_parameters(data, responsibility)

    # Find the clusters
    clusters = get_clusters(data, responsibility)

    # Plot results
    plot_cluster(clusters, mu, sigma)

    # Check for convergence (simplified)
    if np.linalg.norm(old_mu - mu) < epsilon:
        print('Algorithm converged!')
        break